In [1]:
%run ./VA_sentencing_reform_v1_utils.ipynb

52291


In [2]:
reload(incarceration_transitions)
reload(population_simulation)

<module 'spark.sentencing_policy_impact_v1.population_simulation' from '../../spark/sentencing_policy_impact_v1/population_simulation.py'>

In [3]:
from spark.E2E_v2.super_simulation import SuperSimulation
from spark.E2E_v2.incarceration_transitions import IncarceratedTransitions
from spark.E2E_v2.release_transitions import ReleasedTransitions


In [10]:
outflows_data = pd.DataFrame({
    'subgroup': ['affected_population'] * 5,
    'compartment': ['prison_shell', 'prison_shell', 'prison_shell', 'prison_shell', 'prison_shell'],
    'outflow_to': ['supervision', 'supervision', 'supervision', 'supervision', 'supervision'],
    'year': [2020, 2021, 2022, 2023, 2024],
    'total_population': [156,200,200,200,200]
})

transition_data = pd.DataFrame({
    'subgroup': ['affected_population'] * 6,
    'compartment': ['supervision', 'supervision', 'supervision', 'supervision', 'prison', 'release'],
    'outflow_to': ['prison', 'prison', 'prison', 'release', 'prison', 'release'],
    'total_population': [1,2,3,94,1,1],
    'compartment_duration': [1,2,3,3,1,1]
})

transition_data

,subgroup,compartment,outflow_to,total_population,compartment_duration
0,affected_population,supervision,prison,1,1
1,affected_population,supervision,prison,2,2
2,affected_population,supervision,prison,3,3
3,affected_population,supervision,release,94,3
4,affected_population,prison,prison,1,1
5,affected_population,release,release,1,1


In [11]:
user_inputs = {'projection_years': 5, 'policy_year': 2020, 'start_year': 2020}
compartments_architecture = {'prison': IncarceratedTransitions, 'release': ReleasedTransitions, 
                             'prison_shell': None, 'supervision': IncarceratedTransitions}
disaggregation_axes = ['subgroup']
policy_projection = SuperSimulation()
policy_projection.initialize_data(outflows_data, transition_data, compartments_architecture, disaggregation_axes)
policy_projection.set_user_inputs(user_inputs)
projection_results = policy_projection.simulate_baseline(0)

initialization time:  0.006392002105712891
simulation_time:  0.13052797317504883


KeyError: 'release'

In [12]:
results = policy_projection.pop_simulations['baseline'].population_projections
results[results['compartment'] == 'supervision']

,total_population,compartment,year,simulation_group
2020,156.00,supervision,2020,affected_population
2021,354.44,supervision,2021,affected_population
2022,549.32,supervision,2022,affected_population
2023,592.00,supervision,2023,affected_population
2024,592.00,supervision,2024,affected_population


In [ ]:
error = policy_projection.pop_simulations['control'].calculate_transition_error(VA_validation_data)

In [ ]:
print(error)
print(policy_projection.pop_simulations['control'].validation_transition_data)